In [20]:
import numpy as np
import pandas as pd
import matplotlib as mpl
from matplotlib import rc
import matplotlib.pyplot as plt
import matplotlib.figure as fig
import math
import matplotlib.ticker as mtick
from matplotlib.patches import Rectangle

relError = ["1E-9", "1E-8", "1E-7", "1E-6", "1E-5", "1E-4", "1E-3", "1E-2", "1E-1"]

data = ["astro.dat","blast2_p.dat","bump.dat",
        "dpot.dat","eddy.dat","fish.dat",
        "sedov_p.dat","yf17_p.dat","yf17_t.dat"]

df = pd.read_csv("sz_results.csv", index_col=0 ,dtype=str)

df.input_size = df.input_size.astype(float)
df.max_quant_intl = df.max_quant_intl.astype(float)
df.check_radius =  df.check_radius.astype(float)
df.node_count = df.node_count.astype(float)
df.node_count_esti = df.node_count_esti.astype(float)
df.curve_hit = df.curve_hit.astype(float)
df.curve_missed = df.curve_missed.astype(float)
df.hit_ratio = df.hit_ratio.astype(float)
df.tree_size = df.tree_size.astype(float)
df.encode_size = df.encode_size.astype(float)
df.outlier_size = df.outlier_size.astype(float)
df.output_size = df.output_size.astype(float)
df.compression_ratio = df.compression_ratio.astype(float)

In [21]:
# read binary data
import struct
def read_dat_d(d_):
    with open(d_,"rb") as dfile:
        data = dfile.read()
        fmat_string = "="+str(int(len(data)/8))+"d"
        d_data = struct.unpack(fmat_string, data) # decoded double data
    return d_data

def read_dat_f(d_):
    with open(d_,"rb") as dfile:
        data = dfile.read()
        fmat_string = "="+str(int(len(data)/4))+"f"
        d_data = struct.unpack(fmat_string, data) # decoded float data
    return d_data

## hitratio estimation

In [22]:
def hitratio_estimate(fname):
#     print(fname) 
    results = pd.read_csv("sz_results.csv",index_col=0 ,dtype=str)
    data=read_dat_d(str("inputdata/"+fname))
    hitratio_esti = []

    for i in range(0,9): # 9 is the number of error bounds.
        count = 0
        prediction_esti = np.zeros(len(data))
        prediction_esti[0] = data[0]
        prediction_esti[1] = data[1]
        prederror_esti = np.zeros(len(data))
        
#         print(results.errBoundRatio.values == relError[i])
        checkRadius = results.check_radius[(results.data.values == fname) & 
                                           (results.errBoundRatio.values == relError[i])]

        for j in range(2,len(data)):
            prediction_esti[j] = 2*data[j-1]-data[j-2]
            prederror_esti[j] = data[j] - prediction_esti[j]
            if (abs(prederror_esti[j]) < np.float(checkRadius)): # Hit
                count = count+1

        hitratio_esti.append(count/len(data))

    return hitratio_esti

In [24]:
hitratio_estimate("blast2_p.dat")

[0.8896558872305141,
 0.8896558872305141,
 0.8896558872305141,
 0.9119472084024323,
 0.9670242537313433,
 0.9999965450525152,
 0.991499101713654,
 0.991499101713654,
 0.9999965450525152]

In [28]:
def cr_estimate(df, d_, relError):
    input_size = df.input_size[df.data==d_].values[0]
    nc_esti = df.node_count_esti[df.data == d_].values
    hr_esti = hitratio_estimate(d_)

#     at base error bound
#     print(df['errBoundRatio'])
    ts_esti = [df.tree_size[(df['data'] == d_) & (df['errBoundRatio']==relError[0])].values[0]]
    es_esti = [df.encode_size[(df['data'] == d_) & (df['errBoundRatio']==relError[0])].values[0]]
    os_esti = [df.outlier_size[(df['data'] == d_) & (df['errBoundRatio']==relError[0])].values[0]]
    oc_esti = [df.curve_missed[(df['data'] == d_) & (df['errBoundRatio']==relError[0])].values[0]]
    total_esti = [df.output_size[(df['data'] == d_) & (df['errBoundRatio']==relError[0])].values[0]]
    cr_esti = [df.compression_ratio[(df['data'] == d_) & (df['errBoundRatio'] == relError[0])].values[0]]
    
    num_elements = df.curve_missed[df.data==d_].values[0] + df.curve_hit[df.data==d_].values[0]
    for i in range(1,len(relError)): 
        ts_e = ts_esti[0]*(nc_esti[i]/nc_esti[0])
        es_e = es_esti[0]*(np.log2(nc_esti[i])/np.log2(nc_esti[0]))
        oc_e = num_elements*(1-hr_esti[i])
        os_e = oc_e * os_esti[0]/oc_esti[0]
        total_e = ts_e + es_e + os_e
        cr_e = input_size/total_e
        
        ts_esti.append(ts_e)
        es_esti.append(es_e)
        os_esti.append(os_e)
        oc_esti.append(oc_e)
        total_esti.append(total_e)
        cr_esti.append(cr_e)
# [nc_esti,ts_esti,es_esti,os_esti,total_esti,cr_esti]
    return cr_esti

In [35]:
# for name in data:
#     print(name)
#     print(df.compression_ratio[df.data==name].values)
#     print(cr_estimate(df,name, relError))

## sz estimation error

In [10]:
# for i in range(0,len(data)):
# #     print(data[i])
#     orig = df.compression_ratio[df.data == data[i]]
#     hr_esti = Hr_esti.value[Hr_esti.name==data[i]][i]
#     [nc_esti,ts_esti,es_esti,os_esti,total_esti,cr_esti] = cr_estimate(df, hr_esti, data[i], relError)
#     print(np.absolute(np.array(orig) - np.array(cr_esti))/np.array(orig))

## print

In [30]:
# # print hit ratio
# def hit_ratio(Hr_esti,data,relError):
#     rc('text', usetex=True)
#     plt.rc('font',family='Times New Roman')
#     plt.rc('font', size=100)          # controls default text sizes
#     plt.rc('axes', titlesize=90)     # fontsize of the axes title
#     plt.rc('axes', labelsize=90)    # fontsize of the x and y labels
#     plt.rc('xtick', labelsize=90)    # fontsize of the tick labels
#     plt.rc('ytick', labelsize=90)    # fontsize of the tick labels
#     plt.rc('legend', fontsize=90)    # legend fontsize

#     fig, axes  = plt.subplots(nrows=2,ncols=5, sharex=True, sharey=True)
#     fig.set_size_inches(120,60)
#     x = np.arange(1,len(data))
#     x1 = np.array([1,2.5,4,5.5,7])
#     x2 = x1 + 0.6
#     barWidth = 0.5
#     xtick = np.array([r"$\mathbf{10^{-9}}$", r"$\mathbf{10^{-8}}$", r"$\mathbf{10^{-7}}$", r"$\mathbf{10^{-6}}$",
#                       r"$\mathbf{10^{-5}}$", r"$\mathbf{10^{-4}}$", r"$\mathbf{10^{-3}}$", r"$\mathbf{10^{-2}}$", r"$\mathbf{10^{-1}}$"])

#     for i in range(0,len(data)):
# #         print(data[i])
#         hr_esti = Hr_esti.value[Hr_esti.name == data[i]].values[0]
#         axes[int(i/5)][i%5].bar(x1, df.hit_ratio[df.data==data[i]].values[[0,2,4,6,8]],width=barWidth)
#         axes[int(i/5)][i%5].bar(x2, np.array(hr_esti)[[0,2,4,6,8]],width=barWidth)
#         axes[int(i/5)][i%5].set_xticks(x1+0.4)
#         axes[int(i/5)][i%5].set_yticks([0.0,0.2,0.4,0.6,0.8,1.0])
#         axes[int(i/5)][i%5].set_xticklabels(xtick[[0,2,4,6,8]])
#         axes[int(i/5)][i%5].set_yticklabels([r"\textbf{0.0}",r"\textbf{0.2}",r"\textbf{0.4}",r"\textbf{0.6}",r"\textbf{0.8}",r"\textbf{1.0}"])
#         if i == 19:
#             axes[int(i/5)][i%5].legend((r'\textbf{real}',r'\textbf{estimated}'),fancybox=True,loc='right',bbox_to_anchor=(1, -0.3),ncol=1)

#     axes[0][0].set_title(r"\textbf{Astro}")
#     axes[0][1].set_title(r"\textbf{Blast2\_p}")
#     axes[0][2].set_title(r"\textbf{Bump}")
#     axes[0][3].set_title(r"\textbf{Dpot}")
#     axes[0][4].set_title(r"\textbf{Eddy}")
#     axes[1][0].set_title(r"\textbf{Fish}")
#     axes[1][1].set_title(r"\textbf{Sedov}")
#     axes[1][2].set_title(r"\textbf{Yf17\_p}")
#     axes[1][3].set_title(r"\textbf{Yf17\_t}")
   

#     fig.text(0.5, 0.05, r'\textbf{Relative error bound}', ha='center',fontsize=120)
#     fig.text(0.00, 0.5, r'\textbf{HitRatio}', va='center', rotation=90,fontsize=120)

#     plt.tight_layout()
# #     plt.savefig("sz_20_hit_ratio_sdr.pdf")
#     plt.show()
    

In [31]:
# def print_metrics(df,Hr_esti,d_,relError):
#     rc('text', usetex=True)
#     plt.rc('font',family='Times New Roman')
#     plt.rc('font', size=70)          # controls default text sizes
#     plt.rc('axes', titlesize=60)     # fontsize of the axes title
#     plt.rc('axes', labelsize=55)    # fontsize of the x and y labels
#     plt.rc('xtick', labelsize=55)    # fontsize of the tick labels
#     plt.rc('ytick', labelsize=55)    # fontsize of the tick labels
#     plt.rc('legend', fontsize=40)    # legend fontsize
   
#     [nc_esti,ts_esti,es_esti,os_esti,total_esti,cr_esti] = cr_estimate(df, d_, relError)
    
#     x = np.arange(1,10)
#     fig, axearr = plt.subplots(2,3,sharex =True,figsize = (33,23))

#     barWidth = 0.5
#     xtick = np.array([r"$\mathbf{10^{-9}}$", r"$\mathbf{10^{-8}}$", r"$\mathbf{10^{-7}}$", r"$\mathbf{10^{-6}}$",
#                       r"$\mathbf{10^{-5}}$", r"$\mathbf{10^{-4}}$", r"$\mathbf{10^{-3}}$", r"$\mathbf{10^{-2}}$", r"$\mathbf{10^{-1}}$"])

#     x1 = np.array([1,2.5,4,5.5,7])
#     x2 = x1 + 0.6

#     handles = [Rectangle((0,0),1,1)]

#     axearr[0][0].bar(x1,np.array(df.node_count[df['data']==d_].values,dtype=float)[[0,2,4,6,8]],width=barWidth)
#     axearr[0][0].bar(x2,np.array(nc_esti)[[0,2,4,6,8]],width=barWidth)
# #     axearr[0][0].set_yticks([0,1000000,2000000,3000000])
# #     axearr[0][0].set_yticklabels([r"$\mathbf{0}$",r"$\mathbf{1e6}$",r"$\mathbf{2e6}$",r"$\mathbf{3e6}$"])
#     axearr[0][0].set_title(r"\textbf{NodeCount}")
#     axearr[0][0].legend((r'\textbf{real}', r'\textbf{estimated}'),fancybox=True,loc='upper right',ncol=1)
#     plt.setp(axearr[0][0].get_xticklabels(), visible=False)

#     axearr[0][1].bar(x1,np.array(df.tree_size[df['data']==d_].values,dtype=float)[[0,2,4,6,8]],width=barWidth)
#     axearr[0][1].bar(x2,np.array(ts_esti)[[0,2,4,6,8]],width=barWidth)
# #     axearr[0][1].set_yticks([0,10000000,30000000,50000000])
# #     axearr[0][1].set_yticklabels([r"$\mathbf{0}$",r"$\mathbf{1e7}$",r"$\mathbf{3e7}$",r"$\mathbf{5e7}$"])
#     plt.setp(axearr[0][1].get_xticklabels(), visible=False)
# #     axearr[0][1].legend((r'\textbf{real}', r'\textbf{estimated}'),loc='right')
#     axearr[0][1].set_title(r"\textbf{TreeSize}")

#     axearr[0][2].bar(x1,np.array(df.encode_size[df['data']==d_].values,dtype=float)[[0,2,4,6,8]],width=barWidth)
#     axearr[0][2].bar(x2,np.array(es_esti)[[0,2,4,6,8]],width=barWidth)
# #     axearr[0][2].set_yticks([0,10000000,20000000])
# #     axearr[0][2].set_ylim([0,30000000])
# #     axearr[0][2].set_yticklabels([r"$\mathbf{0}$",r"$\mathbf{1e7}$",r"$\mathbf{2e7}$"])
#     plt.setp(axearr[0][2].get_xticklabels(), visible=False)
# #     axearr[0][2].legend((r'\textbf{real}', r'\textbf{estimated}'),loc='right')
#     axearr[0][2].set_title(r"\textbf{EncodeSize}")

#     axearr[1][0].bar(x1,np.array(df.outlier_size[df['data']==d_].values,dtype=float)[[0,2,4,6,8]],width=barWidth)
#     axearr[1][0].bar(x2,np.array(os_esti)[[0,2,4,6,8]],width=barWidth)
# #     axearr[1][0].set_ylim([0,22000000])
# #     axearr[1][0].set_yticks([0,5000000,10000000,15000000,20000000])
# #     axearr[1][0].set_yticklabels([r"$\mathbf{0}$",r"$\mathbf{5e6}$",r"$\mathbf{1e7}$",r"$\mathbf{1.5e7}$",r"$\mathbf{2e7}$"])
#     axearr[1][0].set_xticklabels(xtick[[0,2,4,6,8]])
# #     axearr[1][0].legend((r'\textbf{real}', r'\textbf{estimated}'),loc='right')
#     axearr[1][0].set_title(r"\textbf{OutlierSize}")

#     axearr[1][1].bar(x1,np.array(df.hit_ratio[df['data']==d_].values,dtype=float)[[0,2,4,6,8]],width=barWidth)
#     axearr[1][1].bar(x2,np.array(hr_esti)[[0,2,4,6,8]],width=barWidth)
#     axearr[1][1].set_xticks(x1+0.5)
# #     axearr[1][1].set_yticklabels([r"\textbf{0.00}",r"\textbf{0.25}",r"\textbf{0.50}",r"\textbf{0.75}",r"\textbf{1.00}"])
# #     axearr[1][1].legend((r'\textbf{real}', r'\textbf{estimated}'),loc='right')
#     axearr[1][1].set_title(r"\textbf{HitRatio}")

#     axearr[1][2].bar(x1,np.array(df.compression_ratio[df['data']==d_].values,dtype=float)[[0,2,4,6,8]],width=barWidth)
#     axearr[1][2].bar(x2,np.array(cr_esti)[[0,2,4,6,8]],width=barWidth)
#     axearr[1][2].set_xticklabels(xtick[[0,2,4,6,8]])
# #     axearr[1][2].set_yticks([0,30,60,90,120,150])
# #     axearr[1][2].set_yticklabels([r"\textbf{0}",r"$\mathbf{30}$",r"$\mathbf{60}$",r"$\mathbf{90}$",r"$\mathbf{120}$",r"$\mathbf{150}$"])
# #     axearr[1][2].legend((r'\textbf{real}', r'\textbf{estimated}'),fancybox=True,loc='right',bbox_to_anchor=(1, -0.25),ncol=1)
#     axearr[1][2].set_title(r"\textbf{CompressionRatio}")

#     fig.text(0.5, 0.005, r'\textbf{Relative error bound}', ha='center',fontsize=70)
#     fig.text(0.006, 0.5, r'\textbf{CompressionMetrics}', va='center', rotation=90,fontsize=70)
        
#     plt.tight_layout()
# #     plt.savefig("sz_20_metric_modeling_xgc.pdf")
#     plt.show()

# hr_esti = hitratio_estimate(data[0])
# print_metrics(df,hr_esti,data[0],relError)

### print sz estimation

In [32]:
# def print_sz_cr_esti(data,df, relError):
#     rc('text', usetex=True)
#     plt.rc('font',family='Times New Roman')
#     plt.rc('font', size=100)          # controls default text sizes
#     plt.rc('axes', titlesize=90)     # fontsize of the axes title
#     plt.rc('axes', labelsize=90)    # fontsize of the x and y labels
#     plt.rc('xtick', labelsize=90)    # fontsize of the tick labels
#     plt.rc('ytick', labelsize=90)    # fontsize of the tick labels
#     plt.rc('legend', fontsize=90)    # legend fontsize

#     fig, axes  = plt.subplots(nrows=2,ncols=5, sharex=True)
#     fig.set_size_inches(120,60)
#     x = np.arange(1,10)
#     x1 = np.array([1,2.5,4,5.5,7])
#     x2 = x1 + 0.6
#     barWidth = 0.5
# #     xtick = np.array([r"$\mathbf{10^{-9}}$", r"$\mathbf{10^{-8}}$", r"$\mathbf{10^{-7}}$", r"$\mathbf{10^{-6}}$",
# #                       r"$\mathbf{10^{-5}}$", r"$\mathbf{10^{-4}}$", r"$\mathbf{10^{-3}}$", r"$\mathbf{10^{-2}}$", 
# #                       r"$\mathbf{10^{-1}}$"])

#     for i in range(0,len(data)):
#         hr_esti = hitratio_estimate(data[i])
#         [nc_esti,ts_esti,es_esti,os_esti,total_esti,cr_esti] = cr_estimate(df, data[i], relError)
#         axes[int(i/5)][i%5].bar(x1, df.compression_ratio[df.data==data[i]].values[[0,2,4,6,8]],width=barWidth)
#         axes[int(i/5)][i%5].bar(x2, np.array(cr_esti)[[0,2,4,6,8]],width=barWidth)
#         axes[int(i/5)][i%5].set_xticks(x1+0.4)
#         axes[int(i/5)][i%5].set_xticklabels(xtick[[0,2,4,6,8]])
#         if i == 0:
#             axes[int(i/5)][i%5].legend((r'\textbf{real}',r'\textbf{estimated}'),fancybox=True,loc='upper left',ncol=1)
    
#     axes[0][0].set_title(r"\textbf{Astro}")
#     axes[0][1].set_title(r"\textbf{Blast2\_p}")
#     axes[0][2].set_title(r"\textbf{Bump}")
#     axes[0][3].set_title(r"\textbf{Dpot}")
#     axes[0][4].set_title(r"\textbf{Eddy}")
#     axes[1][0].set_title(r"\textbf{Fish}")
#     axes[1][1].set_title(r"\textbf{Sedov}")
#     axes[1][2].set_title(r"\textbf{Yf17\_p}")
#     axes[1][3].set_title(r"\textbf{Yf17\_t}")
    
#     fig.text(0.5, 0.005, r'\textbf{Relative error bound}', ha='center',fontsize=120)
#     fig.text(0.001, 0.5, r'\textbf{CompressionRatio}', va='center', rotation=90,fontsize=120)
    
#     plt.tight_layout()
#     plt.show()
#     plt.clf()

# print_sz_cr_esti(data,df, relError)

## print sz and zfp original compression ratio

In [33]:
# # print compression ratio
# def print_compressionratio(data,df,Hr_esti, relError):
#     rc('text', usetex=True)
#     plt.rc('font',family='Times New Roman')
#     plt.rc('font', size=100)          # controls default text sizes
#     plt.rc('axes', titlesize=90)     # fontsize of the axes title
#     plt.rc('axes', labelsize=90)    # fontsize of the x and y labels
#     plt.rc('xtick', labelsize=90)    # fontsize of the tick labels
#     plt.rc('ytick', labelsize=90)    # fontsize of the tick labels
#     plt.rc('legend', fontsize=90)    # legend fontsize

#     fig, axes  = plt.subplots(nrows=4,ncols=5, sharex=True)
#     fig.set_size_inches(120,60)
#     x = np.arange(1,10)
#     x1 = np.array([1,2.5,4,5.5,7])
#     x2 = x1 + 0.6
#     barWidth = 0.5
#     xtick = np.array([r"$\mathbf{10^{-9}}$", r"$\mathbf{10^{-8}}$", r"$\mathbf{10^{-7}}$", r"$\mathbf{10^{-6}}$",
#                       r"$\mathbf{10^{-5}}$", r"$\mathbf{10^{-4}}$", r"$\mathbf{10^{-3}}$", r"$\mathbf{10^{-2}}$", 
#                       r"$\mathbf{10^{-1}}$"])

#     for i in range(0,len(data)):
#         axes[int(i/5)][i%5].set_xticks(x1+0.4)
#         axes[int(i/5)][i%5].set_xticklabels(xtick[[0,2,4,6,8]])
#         axes[int(i/5)][i%5].bar(x1, df.compression_ratio[df.data==data[i]].values[[0,2,4,6,8]],width=barWidth,color="tab:blue")
#         axes[int(i/5)][i%5].bar(x1, [0,0,0,0,0],width=barWidth,color="tab:orange")
#         if i == 0:
#             axes[int(i/5)][i%5].legend([r'\textbf{sz}',r'\textbf{zfp}'],fancybox=True,loc='upper left',ncol=1)
    
#     axes[0][0].set_yticks([0,10,20,30])
#     axes[0][0].set_yticklabels([r"\textbf{0}",r"\textbf{10}",r"\textbf{20}",r"\textbf{30}"],color="tab:blue")
#     axes[0][0].set_title(r"\textbf{Astro}")
#     axes2 = axes[0][0].twinx()
#     axes2.bar(x2, df_zfp.compression_ratio[df_zfp.data==data[0]].values[[0,2,4,6,8]],width=barWidth,color="tab:orange")
#     axes2.set_yticks([0,2,4,6,8,10])
#     axes2.set_yticklabels([r"\textbf{0}",r"\textbf{2}",r"\textbf{4}",r"\textbf{6}",r"\textbf{8}",r"\textbf{10}"],color="tab:orange")
    
#     axes[0][1].set_yticks([0,20,40])
#     axes[0][1].set_yticklabels([r"\textbf{0}",r"\textbf{20}",r"\textbf{40}"],color="tab:blue")
#     axes[0][1].set_title(r"\textbf{Blast2\_p}")
#     axes2 = axes[0][1].twinx()
#     axes2.bar(x2, df_zfp.compression_ratio[df_zfp.data==data[1]].values[[0,2,4,6,8]],width=barWidth,color="tab:orange")
#     axes2.set_yticks([0,5,10,15,20])
#     axes2.set_yticklabels([r"\textbf{0}",r"\textbf{5}",r"\textbf{10}",r"\textbf{15}",r"\textbf{20}"],color="tab:orange")
    
#     axes[0][2].set_yticks([0,10,20,30])
#     axes[0][2].set_yticklabels([r"\textbf{0}",r"\textbf{10}",r"\textbf{20}",r"\textbf{30}"],color="tab:blue")
#     axes[0][2].set_title(r"\textbf{Bump}")
#     axes2 = axes[0][2].twinx()
#     axes2.bar(x2, df_zfp.compression_ratio[df_zfp.data==data[2]].values[[0,2,4,6,8]],width=barWidth,color="tab:orange")
#     axes2.set_yticks([0,2,4,6,8])
#     axes2.set_yticklabels([r"\textbf{0}",r"\textbf{2}",r"\textbf{4}",r"\textbf{6}",r"\textbf{8}"],color="tab:orange")
    
#     axes[0][3].set_yticks([0,20,40,60])
#     axes[0][3].set_yticklabels([r"\textbf{0}",r"\textbf{20}",r"\textbf{40}",r"\textbf{60}"],color="tab:blue")
#     axes[0][3].set_title(r"\textbf{Dpot}")
#     axes2 = axes[0][3].twinx()
#     axes2.bar(x2, df_zfp.compression_ratio[df_zfp.data==data[3]].values[[0,2,4,6,8]],width=barWidth,color="tab:orange")
#     axes2.set_yticks([0,2,4,6,8])
#     axes2.set_yticklabels([r"\textbf{0}",r"\textbf{2}",r"\textbf{4}",r"\textbf{6}",r"\textbf{8}"],color="tab:orange")
    
#     axes[0][4].set_yticks([0,10,20,30])
#     axes[0][4].set_yticklabels([r"\textbf{0}",r"\textbf{10}",r"\textbf{20}",r"\textbf{30}"],color="tab:blue")
#     axes[0][4].set_title(r"\textbf{Eddy}")
#     axes2 = axes[0][4].twinx()
#     axes2.bar(x2, df_zfp.compression_ratio[df_zfp.data==data[4]].values[[0,2,4,6,8]],width=barWidth,color="tab:orange")
#     axes2.set_yticks([0,5,10,15])
#     axes2.set_yticklabels([r"\textbf{0}",r"\textbf{5}",r"\textbf{10}",r"\textbf{15}"],color="tab:orange")
    
#     axes[1][0].set_yticks([0,20,40])
#     axes[1][0].set_yticklabels([r"\textbf{0}",r"\textbf{20}",r"\textbf{40}"],color="tab:blue")
#     axes[1][0].set_title(r"\textbf{Fish}")
#     axes2 = axes[1][0].twinx()
#     axes2.bar(x2, df_zfp.compression_ratio[df_zfp.data==data[5]].values[[0,2,4,6,8]],width=barWidth,color="tab:orange")
#     axes2.set_yticks([0,10,20,30])
#     axes2.set_yticklabels([r"\textbf{0}",r"\textbf{10}",r"\textbf{20}",r"\textbf{30}"],color="tab:orange")
    
#     axes[1][1].set_yticks([0,10,20,30])
#     axes[1][1].set_yticklabels([r"\textbf{0}",r"\textbf{10}",r"\textbf{20}",r"\textbf{30}"],color="tab:blue")
#     axes[1][1].set_title(r"\textbf{Sedov}")
#     axes2 = axes[1][1].twinx()
#     axes2.bar(x2, df_zfp.compression_ratio[df_zfp.data==data[6]].values[[0,2,4,6,8]],width=barWidth,color="tab:orange")
#     axes2.set_yticks([0,2.5,5.0,7.5])
#     axes2.set_yticklabels([r"\textbf{0}",r"\textbf{2.5}",r"\textbf{5.0}",r"\textbf{7.5}"],color="tab:orange")
    
#     axes[1][2].set_yticks([0,10,20,30])
#     axes[1][2].set_yticklabels([r"\textbf{0}",r"\textbf{10}",r"\textbf{20}",r"\textbf{30}"],color="tab:blue")
#     axes[1][2].set_title(r"\textbf{Yf17\_p}")
#     axes2 = axes[1][2].twinx()
#     axes2.bar(x2, df_zfp.compression_ratio[df_zfp.data==data[7]].values[[0,2,4,6,8]],width=barWidth,color="tab:orange")
#     axes2.set_yticks([0,2.5,5.0,7.5])
#     axes2.set_yticklabels([r"\textbf{0}",r"\textbf{2.5}",r"\textbf{5.0}",r"\textbf{7.5}"],color="tab:orange")
    
#     axes[1][3].set_yticks([0,10,20,30])
#     axes[1][3].set_yticklabels([r"\textbf{0}",r"\textbf{10}",r"\textbf{20}",r"\textbf{30}"],color="tab:blue")
#     axes[1][3].set_title(r"\textbf{Yf17\_t}")
#     axes2 = axes[1][3].twinx()
#     axes2.bar(x2, df_zfp.compression_ratio[df_zfp.data==data[8]].values[[0,2,4,6,8]],width=barWidth,color="tab:orange")
#     axes2.set_yticks([0,2.5,5.0,7.5])
#     axes2.set_yticklabels([r"\textbf{0}",r"\textbf{2.5}",r"\textbf{5.0}",r"\textbf{7.5}"],color="tab:orange")
    
#     axes[1][4].set_yticks([0,25,50,75])
#     axes[1][4].set_yticklabels([r"\textbf{0}",r"\textbf{25}",r"\textbf{50}",r"\textbf{75}"],color="tab:blue")
#     axes[1][4].set_title(r"\textbf{NWChem}")
#     axes2 = axes[1][4].twinx()
#     axes2.bar(x2, df_zfp.compression_ratio[df_zfp.data==data[9]].values[[0,2,4,6,8]],width=barWidth,color="tab:orange")
#     axes2.set_yticks([0,5,10,15])
#     axes2.set_yticklabels([r"\textbf{0}",r"\textbf{5}",r"\textbf{10}",r"\textbf{15}"],color="tab:orange")
    
#     axes[2][0].set_yticks([0,5,10,15,20])
#     axes[2][0].set_yticklabels([r"\textbf{0}",r"\textbf{5}",r"\textbf{10}",r"\textbf{15}",r"\textbf{20}"],color="tab:blue")
#     axes[2][0].set_title(r"\textbf{CESM\_ATM}")
#     axes2 = axes[2][0].twinx()
#     axes2.bar(x2, df_zfp.compression_ratio[df_zfp.data==data[10]].values[[0,2,4,6,8]],width=barWidth,color="tab:orange")
#     axes2.set_yticks([0,5,10])
#     axes2.set_yticklabels([r"\textbf{0}",r"\textbf{5}",r"\textbf{10}"],color="tab:orange")
    
#     axes[2][1].set_yticks([0,20,40])
#     axes[2][1].set_yticklabels([r"\textbf{0}",r"\textbf{20}",r"\textbf{40}"],color="tab:blue")
#     axes[2][1].set_title(r"\textbf{Hurrican\_ISABEL}")
#     axes2 = axes[2][1].twinx()
#     axes2.bar(x2, df_zfp.compression_ratio[df_zfp.data==data[11]].values[[0,2,4,6,8]],width=barWidth,color="tab:orange")
#     axes2.set_yticks([0,10,20])
#     axes2.set_yticklabels([r"\textbf{0}",r"\textbf{10}",r"\textbf{20}"],color="tab:orange")
    
#     axes[2][2].set_yticks([0,10,20,30])
#     axes[2][2].set_yticklabels([r"\textbf{0}",r"\textbf{10}",r"\textbf{20}",r"\textbf{30}"],color="tab:blue")
#     axes[2][2].set_title(r"\textbf{QMCPACK}")
#     axes2 = axes[2][2].twinx()
#     axes2.bar(x2, df_zfp.compression_ratio[df_zfp.data==data[12]].values[[0,2,4,6,8]],width=barWidth,color="tab:orange")
#     axes2.set_yticks([0,2,4])
#     axes2.set_yticklabels([r"\textbf{0}",r"\textbf{2}",r"\textbf{4}"],color="tab:orange")
    
#     axes[2][3].set_yticks([0,40,80,120])
#     axes[2][3].set_yticklabels([r"\textbf{0}",r"\textbf{40}",r"\textbf{80}",r"\textbf{120}"],color="tab:blue")
#     axes[2][3].set_title(r"\textbf{Brown}")
#     axes2 = axes[2][3].twinx()
#     axes2.bar(x2, df_zfp.compression_ratio[df_zfp.data==data[13]].values[[0,2,4,6,8]],width=barWidth,color="tab:orange")
#     axes2.set_yticks([0,5,10])
#     axes2.set_yticklabels([r"\textbf{0}",r"\textbf{5}",r"\textbf{10}"],color="tab:orange")
    
#     axes[2][4].set_yticks([0,10,20,30])
#     axes[2][4].set_yticklabels([r"\textbf{0}",r"\textbf{10}",r"\textbf{20}",r"\textbf{30}"],color="tab:blue")
#     axes[2][4].set_title(r"\textbf{SCALE\_LETKF}")
#     axes2 = axes[2][4].twinx()
#     axes2.bar(x2, df_zfp.compression_ratio[df_zfp.data==data[14]].values[[0,2,4,6,8]],width=barWidth,color="tab:orange")
#     axes2.set_yticks([0,2,4,6])
#     axes2.set_yticklabels([r"\textbf{0}",r"\textbf{2}",r"\textbf{4}",r"\textbf{6}"],color="tab:orange")
    
#     axes[3][0].set_yticks([0,5,10,15])
#     axes[3][0].set_yticklabels([r"\textbf{0}",r"\textbf{5}",r"\textbf{10}",r"\textbf{15}"],color="tab:blue")
#     axes[3][0].set_title(r"\textbf{NYX}")
#     axes2 = axes[3][0].twinx()
#     axes2.bar(x2, df_zfp.compression_ratio[df_zfp.data==data[15]].values[[0,2,4,6,8]],width=barWidth,color="tab:orange")
#     axes2.set_yticks([0,2,4])
#     axes2.set_yticklabels([r"\textbf{0}",r"\textbf{2}",r"\textbf{4}"],color="tab:orange")
    
#     axes[3][1].set_yticks([0,5,10,15,20])
#     axes[3][1].set_yticklabels([r"\textbf{0}",r"\textbf{10}",r"\textbf{15}",r"\textbf{20}"],color="tab:blue")
#     axes[3][1].set_title(r"\textbf{EXAALT}")
#     axes2 = axes[3][1].twinx()
#     axes2.bar(x2, df_zfp.compression_ratio[df_zfp.data==data[16]].values[[0,2,4,6,8]],width=barWidth,color="tab:orange")
#     axes2.set_yticks([0,1,2,3])
#     axes2.set_yticklabels([r"\textbf{0}",r"\textbf{1}",r"\textbf{2}",r"\textbf{3}"],color="tab:orange")
    
#     axes[3][2].set_yticks([0,5,10,15,20])
#     axes[3][2].set_yticklabels([r"\textbf{0}",r"\textbf{5}",r"\textbf{10}",r"\textbf{15}",r"\textbf{20}"],color="tab:blue")
#     axes[3][2].set_title(r"\textbf{HACC}")
#     axes2 = axes[3][2].twinx()
#     axes2.bar(x2, df_zfp.compression_ratio[df_zfp.data==data[17]].values[[0,2,4,6,8]],width=barWidth,color="tab:orange")
#     axes2.set_yticks([0,1,2,3])
#     axes2.set_yticklabels([r"\textbf{0}",r"\textbf{1}",r"\textbf{2}",r"\textbf{3}"],color="tab:orange")
    
#     axes[3][3].set_yticks([0,50,100,150])
#     axes[3][3].set_yticklabels([r"\textbf{0}",r"\textbf{50}",r"\textbf{100}",r"\textbf{150}"],color="tab:blue")
#     axes[3][3].set_title(r"\textbf{XGC}")
#     axes2 = axes[3][3].twinx()
#     axes2.bar(x2, df_zfp.compression_ratio[df_zfp.data==data[18]].values[[0,2,4,6,8]],width=barWidth,color="tab:orange")
#     axes2.set_yticks([0,2,4,6])
#     axes2.set_yticklabels([r"\textbf{0}",r"\textbf{2}",r"\textbf{4}",r"\textbf{6}"],color="tab:orange")
    
#     axes[3][4].set_yticks([0,20,40,60])
#     axes[3][4].set_yticklabels([r"\textbf{0}",r"\textbf{20}",r"\textbf{40}",r"\textbf{60}"],color="tab:blue")
#     axes[3][4].set_title(r"\textbf{S3D}")
#     axes2 = axes[3][4].twinx()
#     axes2.bar(x2, df_zfp.compression_ratio[df_zfp.data==data[19]].values[[0,2,4,6,8]],width=barWidth,color="tab:orange")
#     axes2.set_yticks([0,10,20])
#     axes2.set_yticklabels([r"\textbf{0}",r"\textbf{10}",r"\textbf{20}"],color="tab:orange")
    

#     fig.text(0.5, 0.005, r'\textbf{Relative error bound}', ha='center',fontsize=120)
#     fig.text(0.001, 0.5, r'\textbf{CompressionRatio}', va='center', rotation=90,fontsize=120)
    
#     plt.tight_layout()
#     plt.savefig("sz_20_compression_ratio_orig.pdf")
#     plt.show()
#     plt.clf()
    
# print_compressionratio(data,df,Hr_esti, relError)

### print sz and zfp compression ratio estimation error

In [34]:
# import seaborn as sns

# sns.set()
# current_palette = sns.color_palette("muted")
# sns.set_style("white")

# def print_compressionratio(data,df,Hr_esti, relError):
#     rc('text', usetex=True)
#     plt.rc('font',family='Times New Roman')
#     plt.rc('font', size=100)          # controls default text sizes
#     plt.rc('axes', titlesize=90)     # fontsize of the axes title
#     plt.rc('axes', labelsize=90)    # fontsize of the x and y labels
#     plt.rc('xtick', labelsize=90)    # fontsize of the tick labels
#     plt.rc('ytick', labelsize=90)    # fontsize of the tick labels
#     plt.rc('legend', fontsize=90)    # legend fontsize

#     fig, axes  = plt.subplots(nrows=4,ncols=5, sharex=True)
#     fig.set_size_inches(120,60)
#     x = np.arange(1,10)
#     x1 = np.array([1,2.5,4,5.5,7])
#     x2 = x1 + 0.6
#     barWidth = 0.5
#     xtick = np.array([r"$\mathbf{10^{-9}}$", r"$\mathbf{10^{-8}}$", r"$\mathbf{10^{-7}}$", r"$\mathbf{10^{-6}}$",
#                       r"$\mathbf{10^{-5}}$", r"$\mathbf{10^{-4}}$", r"$\mathbf{10^{-3}}$", r"$\mathbf{10^{-2}}$", 
#                       r"$\mathbf{10^{-1}}$"])
#     sz_error = [[0.,0.98437616,0.98171899,0.93891496,0.56030423,0.10893041,0.08020149,0.02927784,0.02982557],
#                 [0.,0.97913777,0.9844525,0.98153776,0.86145837,0.78112133,0.80257129,0.53264579,0.35984327],
#                 [0.,0.97824784,0.98300912,0.93218617,0.81048008,0.56510573,0.55253464,0.69011267,0.4632481],
#                 [0.,1.,1.,1.,0.02485981,0.89780326,2.091189,2.12998206,0.24029243],
#                 [0.,1.,1.,0.90327695,0.62526999,0.40747475,0.38429947,0.27740585,0.2401439],
#                 [0.,0.99326707,0.96547381,0.7329181,0.07852676,0.46133743,0.52683499,0.71478953,1.00409726],
#                 [0.,0.99125254,0.98814359,0.90749926,0.43946081,0.19483149,0.05150451,0.39587213,0.25305698],
#                 [0.,1.,1.,0.84390172,0.58167454,0.29317174,0.23181956,0.28186116,0.34739202],
#                 [0.,1.,1.,1.,0.70759582,0.03061082,0.0920035,0.11551603,0.12864116],
#                 [0.,0.12802492,0.24535682,0.40488604,0.53012983,0.53944882,0.40803633,0.42474304,0.71247035],
#                 [0.,0.01432497,0.00138705,0.0764258,0.17254825,0.14782051,0.10833703,0.55569235,0.13398449],
#                 [0.,0.07181672,0.2725639,0.251898,0.32295392,0.42245442,0.78120103,1.54755924,1.56415898],
#                 [0.,0.24925329,0.98685495,0.89712473,0.76407233,0.65252972,0.35367662,0.04838948,0.44330483],
#                 [0.,0.00237678,0.04678189,0.04808242,0.14107769,0.70097143,0.56500363,0.56499327,0.56499225],
#                 [0.,0.02571944,0.05601993,0.06834408,0.21392792,0.32770341,0.72217092,0.52083471,0.52217548],
#                 [0.,0.04895962,0.06171305,0.15339675,0.20397873,0.39217913,0.50778106,0.52921681,0.52939759],
#                 [0.,1.,1.,1.,3.54896019,4.95746632,2.68262718,3.57418531,2.37547386],
#                 [0.,0.44022877,1.54821081,1.5328656,1.38571269,1.2458571,0.88764916,0.45827156,0.24553036],
#                 [0.,0.67455118,0.77742876,1.18836252,1.29176739,1.20515285,0.84049527,0.24418878,0.23861659],
#                 [0.,0.19837734,0.33850434,0.471216,0.52859871,0.4631795,0.09187052,0.55214266,0.55408153]]
    
#     esti_err = [[0.,0.02611165, 0.11581205, 0.16905996  ,0.14772282],
#                 [0.,0.26669345, 0.33900687, 0.54263811  ,0.57025919],
#                 [0.,0.01449478, 0.03238649, 0.11063518  ,0.23789727],
#                 [0.,0.07891689, 0.07909762, 0.14656524  ,0.19986196],
#                 [0.,0.01841631, 0.06648632, 0.07092586  ,1.84861057],
#                 [0.,0.81442559, 0.81083671, 0.81490821  ,0.82497682],
#                 [0.,0.0751294 , 0.1514824 , 0.21273609  ,0.18260219],
#                 [0.,0.05516392, 0.12886911, 0.17936117  ,0.29020929],
#                 [0.,0.0103815 , 0.08468523, 0.18509182  ,0.21223684],
#                 [0.,0.39188242, 0.49188958, 0.60164844  ,0.8013849 ],
#                 [0.,0.50602277, 2.41017379, 2.44036697  ,1.36111111],
#                 [0.,0.79434114, 0.77592329, 0.71221124  ,0.22218755],
#                 [0.,0.12011034, 0.1345106 , 0.22042073  ,0.19565217],
#                 [0.,0.0174991 , 0.0204284 , 0.16087706  ,0.04732273],
#                 [0.,0.07637932, 0.19108365, 0.2793723   ,0.33496584],
#                 [0.,0.00967365, 0.05080405, 0.08779139  ,0.22040047],
#                 [0.,0.00274666, 0.02660062, 0.0103444   ,0.07209029],
#                 [0.,0.0156173,  0.0163376, 0.07108416 ,0.09112263],
#                 [0.,0.16825769, 0.19526724,  0.34061922, 0.63338259],
#                 [0.,5.61987552e1, 2.86731251e-01, 2.97370023e-01,2.68776485e-01]]
    
#     for i in range(0,len(data)):
#         axes[int(i/5)][i%5].set_xticks(x1+0.4)
#         axes[int(i/5)][i%5].set_xticklabels(xtick[[0,2,4,6,8]])
#         axes[int(i/5)][i%5].bar(x1, np.array(sz_error[i])[[0,2,4,6,8]],width=barWidth)
#         axes[int(i/5)][i%5].bar(x2, esti_err[i],width=barWidth)
#         axes[int(i/5)][i%5].set_ylim(0,1.1)
#         axes[int(i/5)][i%5].set_yticks([0,0.2,0.4,0.6,0.8,1.0])
#         axes[int(i/5)][i%5].set_yticklabels([r"\textbf{0}",r"\textbf{20}",r"\textbf{40}",r"\textbf{60}",r"\textbf{80}",r"\textbf{100}"])
#         if i == 0:
#             axes[int(i/5)][i%5].legend([r'\textbf{sz}',r'\textbf{zfp}'],fancybox=True,loc='upper right',ncol=1)
        
#     axes[0][0].set_title(r"\textbf{Astro}")
#     axes[0][1].set_title(r"\textbf{Blast2\_p}")
#     axes[0][2].set_title(r"\textbf{Bump}")
#     axes[0][3].set_title(r"\textbf{Dpot}")
#     axes[0][4].set_title(r"\textbf{Eddy}")
#     axes[1][0].set_title(r"\textbf{Fish}")
#     axes[1][1].set_title(r"\textbf{Sedov}")
#     axes[1][2].set_title(r"\textbf{Yf17\_p}")
#     axes[1][3].set_title(r"\textbf{Yf17\_t}")
#     axes[1][4].set_title(r"\textbf{NWChem}")
#     axes[2][0].set_title(r"\textbf{CESM\_ATM}")
#     axes[2][1].set_title(r"\textbf{Hurrican\_ISABEL}")
#     axes[2][2].set_title(r"\textbf{QMCPACK}")
#     axes[2][3].set_title(r"\textbf{Brown}")
#     axes[2][4].set_title(r"\textbf{SCALE\_LETKF}")
#     axes[3][0].set_title(r"\textbf{NYX}")
#     axes[3][1].set_title(r"\textbf{EXAALT}")
#     axes[3][2].set_title(r"\textbf{HACC}")
#     axes[3][3].set_title(r"\textbf{XGC}")
#     axes[3][4].set_title(r"\textbf{S3D}")
    
#     fig.text(0.5, 0.005, r'\textbf{Relative error bound}', ha='center',fontsize=120)
#     fig.text(0.001, 0.5, r'\textbf{Relative estimation error($\%$)}', va='center', rotation=90,fontsize=120)
    
#     plt.tight_layout()
#     plt.savefig("compression_ratio_esti_err_sz_zfp.pdf")
#     plt.show()
#     plt.clf()
    
# print_compressionratio(data,df,Hr_esti, relError)